In [ ]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Mounting the google drive
from google.colab import drive
drive.mount('/content/drive')

# This changes the working folder to the TPX3 folder.
os.chdir('/content/drive/MyDrive/TPX3')

Mounted at /content/drive


In [ ]:
def calculate_energy(tottime, cal_val, lin=False):
    """
    Function that uses given values for calibration to convert from ToT to energy.
    """
    a = cal_val[0]
    b = cal_val[1]
    c = cal_val[2]
    t = cal_val[3]
    y = tottime
    if not lin:
        # print('use non-lin')
        if a == 0:
            print("Error: a = 0")
            return 0
        elif (
            np.sqrt(
                a * a * t * t
                + 2 * a * b * t
                + 4 * a * c
                - 2 * a * t * y
                + b * b
                - 2 * b * y
                + y * y
            )
            + y
            == a * t + b
        ):
            print("Error: second condition not ok")
            return 0
        else:
            # If borh conditions are met, calculate energy from tot
            return float(
                (
                    np.sqrt(
                        a * a * t * t
                        + 2 * a * b * t
                        + 4 * a * c
                        - 2 * a * t * y
                        + b * b
                        - 2 * b * y
                        + y * y
                    )
                    + a * t
                    - b
                    + y
                )
                / (2 * a)
            )
    else:
        # use linear
        if a == 0:
            print("Error: a = 0")
            return 0
        else:
            return float((y - b) / a)

In [ ]:
def cluster_npy_text(filename, cal_val, lin=False):
    """
    Function that saves the cluster data to a .txt file.
    """
    try:
        print("save cluster information to text file")
        tic = time.time()

        clusterdata = np.load(f"./{filename}/{filename}_clusters.npy")
        with open(f"./{filename}/{filename}_clusterstat.txt", "w+") as f_cluster:
            with open(f"./{filename}/{filename}_clusterdata.txt", "w+") as f:
                f.write("Clusterdata\n\n")
                f.write(
                    "index    xPos    yPos    ToATime         ToTTime        ToTEnergy\n"
                )
                cluster_index = 1
                max_cluster_size = 0
                ndx = 0

                e_tot = 0.0
                t_tot = 0.0
                x_tot = 0.0
                y_tot = 0.0
                times = []
                cluster_size = 0

                # new clustermatrix
                # index, clustersize, tot, e (res)
                cluster_matrix = np.zeros((len(clusterdata), 4))

                for i in range(len(clusterdata)):
                    if int(cluster_index) == int(clusterdata[i, 5]):
                        # print(f"pix {i} belongs to cluster {cluster_index}")
                        e = calculate_energy(clusterdata[i, 4], cal_val, lin=lin)
                        f.write(
                            f"{int(clusterdata[i, 0]):5d} {int(clusterdata[i, 1]):7d} "
                            f"{int(clusterdata[i, 2]):7d} {int(clusterdata[i, 3]):14.4f} "
                            f"{int(clusterdata[i, 4]):8d} {e:8f}\n"
                        )
                        # add values for calculation
                        e_tot += e
                        t_tot += clusterdata[i, 4]
                        x_tot += clusterdata[i, 1]
                        y_tot += clusterdata[i, 2]
                        times += [clusterdata[i, 3]]
                        cluster_size += 1
                    else:
                        # save cluster so far
                        x_avg = x_tot / cluster_size
                        y_avg = y_tot / cluster_size
                        if cluster_size > max_cluster_size:
                            max_cluster_size = cluster_size
                        # calculate time difference (window)
                        t_min = min(times)
                        t_max = max(times)
                        d_t = t_max - t_min
                        f.write(
                            f"({x_avg:.1f}, {y_avg:.1f}), E = {e_tot:.1f}, Ttot = {t_tot:.1f}, "
                            f"clustersize = {cluster_size:d}, time window = {d_t:.1f}\n\n"
                        )

                        #                     0                  1             2
                        #                   size               energy         tot
                        f_cluster.write(
                            f"{cluster_size:04d}  {e_tot:08.3f}  {t_tot:08.3f}\n"
                        )

                        cluster_matrix[ndx, :] = [ndx, cluster_size, t_tot, e]
                        ndx += 1

                        # start new cluster
                        e = calculate_energy(clusterdata[i, 4], cal_val, lin=lin)
                        f.write(
                            f"{int(clusterdata[i, 0]):5d} {int(clusterdata[i, 1]):7d} "
                            f"{int(clusterdata[i, 2]):7d} {int(clusterdata[i, 3]):14.4f} "
                            f"{int(clusterdata[i, 4]):8d} {e:8f}\n"
                        )

                        e_tot = e
                        t_tot = clusterdata[i, 4]
                        x_tot = clusterdata[i, 1]
                        y_tot = clusterdata[i, 2]
                        times = [clusterdata[i, 3]]
                        cluster_size = 1

                        cluster_index += 1
                x_avg = x_tot / cluster_size
                y_avg = y_tot / cluster_size
                if cluster_size > max_cluster_size:
                    max_cluster_size = cluster_size
                # calculate time difference (window)
                t_min = min(times)
                t_max = max(times)
                d_t = t_max - t_min
                f.write(
                    f"({x_avg:.1f}, {y_avg:.1f}), E = {e_tot:.1f}, Ttot = {t_tot:.1f}, "
                    f"clustersize = {cluster_size:d}, time window = {d_t:.1f}\n\n"
                )
                f_cluster.write(f"{cluster_size:04d}  {e_tot:08.3f}  {t_tot:08.3f}\n")
                f.write(f"\nMaximum cluster size: {max_cluster_size} pixels.\n")
                cluster_matrix[ndx, :] = [ndx, cluster_size, t_tot, e]
                ndx += 1
                np.save(f"{filename}/{filename}_clustermatrix.npy")

        toc = time.time()
        dt = time.gmtime(toc - tic)
        print(f'converting to text took {time.strftime("%H:%M:%S", dt)}')
    except Exception as e:
        print(e)

In [ ]:
def plot_window(filename, t_window, verbose_level=1):
    """
    Function that plots time windows of raw pixel data.
    """
    print("Plotting frames")
    # load npy file
    pixdat = np.load(f"./{filename}/{filename}_pixeldata.npy")
    # find minimum and maximum time
    TmaxLine = max(pixdat, key=lambda x: x[3])
    TminLine = min(pixdat, key=lambda x: x[3])
    print(TmaxLine)
    print(TminLine)
    # make gif over time
    i = 0  # frame counter
    t_min = int(TminLine[3])
    t_max = int(TmaxLine[3])
    interval = int(t_window / 2)
    num_frames = int((t_max - t_min) / interval)
    print(f"generating {num_frames} frames")
    for t in range(t_min, t_max, interval):
        frame_pixels = []
        for pix in pixdat:
            if (t - t_window / 2) < pix[3] < (t + t_window / 2):
                frame_pixels.append(pix)

        grid = np.zeros((256, 256))
        for pix in frame_pixels:
            grid[int(pix[1]), int(pix[2])] = pix[4]

        fig = plt.figure(figsize=[10, 10], dpi=600)
        ax = fig.add_subplot(111)
        plt.imshow(grid)
        plt.colorbar()
        plt.clim(0, 10)
        props = ""
        ax.text(
            0.05,
            1.1,
            f"t= {t:10d} ns",
            transform=ax.transAxes,
            fontsize=14,
            verticalalignment="top",
        )
        plt.savefig(f"./{filename}/window_frames/XY_frame{i:03d}.png")
        plt.close()
        i += 1

In [ ]:
cluster_npy_text("241Am_60min_clustermatrix",[1,0,0,0], lin=False)

save cluster information to text file
[Errno 2] No such file or directory: './241Am_60min_clustermatrix/241Am_60min_clustermatrix_clusters.npy'


In [ ]:
y = np.load('/content/drive/MyDrive/TPX3/241Am_60min/241Am_60min_clusters.npy')
print(y)
ToT = np.delete(y, [0,1,2,4], 1)

#print(ToT)
#Data_ToT = ToT.T
#print(Data_ToT)
 

[[0.00000000e+00 1.71000000e+02 9.90000000e+01 1.60575000e+05
  4.60000000e+01 1.00000000e+00]
 [1.00000000e+00 2.55000000e+02 9.60000000e+01 1.78221875e+05
  3.20000000e+01 2.00000000e+00]
 [2.00000000e+00 2.55000000e+02 9.50000000e+01 1.78235938e+05
  2.60000000e+01 2.00000000e+00]
 ...
 [5.64933700e+06 1.43000000e+02 1.44000000e+02 3.59943127e+12
  3.10000000e+01 4.85466600e+06]
 [5.64933500e+06 6.30000000e+01 1.00000000e+02 3.59943141e+12
  6.00000000e+01 4.85466700e+06]
 [5.64933600e+06 6.90000000e+01 1.03000000e+02 3.59943141e+12
  6.50000000e+01 4.85466800e+06]]


In [ ]:
x = np.load('/content/drive/MyDrive/TPX3/241Am_60min/241Am_60min_clusters.npy')
print(x)

FileNotFoundError: ignored

In [ ]:
import pandas as pd

data = pd.read_csv('ASTAR_TSP.txt')
print(data)
rho_gcm = 2.329
d_e_cm = 0.01
data['E dep MeV'] = data['Total_Stp._Pow._MeV_cm2/g']*0.02329

     Kinetic_Energy_MeV   Total_Stp._Pow._MeV_cm2/g
0                0.0010                      121.30
1                0.0015                      134.20
2                0.0020                      144.70
3                0.0025                      153.90
4                0.0030                      162.40
..                  ...                         ...
116            800.0000                       14.45
117            850.0000                       13.91
118            900.0000                       13.43
119            950.0000                       13.00
120           1000.0000                       12.61

[121 rows x 2 columns]


KeyError: ignored

In [ ]:
import pandas as pd

data = pd.read_csv('ASTAR_TSP.txt', delim_whitespace = True)
print(data)
rho_gcm = 2.329
d_e_cm = 0.01


             Kinetic  Total
Energy          Stp.   Pow.
MeV              MeV  cm2/g
1.000E-03  1.213E+02    NaN
1.500E-03  1.342E+02    NaN
2.000E-03  1.447E+02    NaN
...              ...    ...
8.000E+02  1.445E+01    NaN
8.500E+02  1.391E+01    NaN
9.000E+02  1.343E+01    NaN
9.500E+02  1.300E+01    NaN
1.000E+03  1.261E+01    NaN

[123 rows x 2 columns]
